In [ ]:
# import os
# import csv
import sys
from datetime import datetime
import pandas as pd
# import json
from kafka import KafkaProducer
from pathlib import Path
# import logging
from loguru import logger
from typing import Any, Dict
import pytz

current_timezone = pytz.timezone('Europe/Moscow')

# Конфигурация
# KAFKA_BROKER = 'localhost:9092'
# KAFKA_TOPIC = 'csv_data_topic'
# CSV_DIRECTORY = '../data/TelecomX/telecom100k/'

# logger = logging.getLogger(__name__)
# logging.basicConfig(level=logging.INFO)
# logger.add(sys.stdout, colorize=True, format="<green>{time}</green> <level>{message}</level>")
# logger.add("netflow_producer.log")


2

In [68]:
from pydantic_settings import BaseSettings, SettingsConfigDict

class Settings(BaseSettings):
    model_config = SettingsConfigDict(env_ignore_empty=True, env_file='.env', env_file_encoding='utf-8')

    current_time: str = 'Europe/Moscow'
    kafka_broker: str = 'localhost:9092'
    kafka_topic: str = 'csv_data_topic'
    csv_directory: str = '../data/TelecomX/telecom100k/'
    log_file: str = "logs/netflow_producer.log"
    time_pointer_file: str = 'logs/time_pointer.txt'


settings = Settings()
KAFKA_BROKER = settings.kafka_broker
KAFKA_TOPIC = settings.kafka_topic
CSV_DIRECTORY = settings.csv_directory

# LOG_FILE = settings['log_

In [ ]:
logger.add(settings.log_file)

3

In [55]:
def get_kafka_producer() -> KafkaProducer:
    """
    Инициализируем KafkaProducer с базовыми настройками.
    В продакшене нужно добавить обработку ошибок подключения, настройки безопасности и т.д.
    """
    try:
        producer = KafkaProducer(
            bootstrap_servers=KAFKA_BROKER,
            # value_serializer=lambda v: json.dumps(v, allow_nan=False).encode('utf-8'),
            value_serializer=lambda v: str(v).encode('utf-8'),
            retries=5,               # Повторная отправка при сбоях
            linger_ms=10,            # Небольшая задержка перед отправкой
            max_request_size=1048576 # Ограничение размера запроса (1MB)
        )
        logger.info("KafkaProducer успешно инициализирован!")
        return producer
    except Exception as e:
        logger.error(f"Ошибка инициализации KafkaProducer: {e}")
        raise

In [56]:
r0_logs = sorted(Path(CSV_DIRECTORY).glob('psx_66.1_*.txt'))
r1_logs = sorted(Path(CSV_DIRECTORY).glob('psx_66.2_*.txt'))
r2_logs = sorted(Path(CSV_DIRECTORY).glob('psx_66.3_*.txt'))
r3_logs = sorted(Path(CSV_DIRECTORY).glob('psx_62.0_*.csv'))
r4_logs = sorted(Path(CSV_DIRECTORY).glob('psx_69.0_*.csv'))
r5_logs = sorted(Path(CSV_DIRECTORY).glob('psx_65.0_*.csv'))

# r3_logs[0:10]

In [57]:
# csv_file1 = Path(CSV_DIRECTORY+'psx_65.0_2024-01-04 08:20:00.csv');
csv_file0 = r0_logs[0]
df0 = pd.read_csv(csv_file0, parse_dates=['StartSession','EndSession'],sep='|').rename(columns={"Duartion": "Duration"})
df0['StartSession'] = df0['StartSession'].dt.tz_localize('Etc/GMT-5').dt.tz_convert('Europe/Moscow')
df0['EndSession'] = df0['EndSession'].dt.tz_localize('Etc/GMT-5').dt.tz_convert('Europe/Moscow')
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1564 entries, 0 to 1563
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype                        
---  ------        --------------  -----                        
 0   IdSession     1564 non-null   int64                        
 1   IdPSX         1564 non-null   int64                        
 2   IdSubscriber  1564 non-null   int64                        
 3   StartSession  1564 non-null   datetime64[ns, Europe/Moscow]
 4   EndSession    20 non-null     datetime64[ns, Europe/Moscow]
 5   Duration      1564 non-null   int64                        
 6   UpTx          1564 non-null   int64                        
 7   DownTx        1564 non-null   int64                        
dtypes: datetime64[ns, Europe/Moscow](2), int64(6)
memory usage: 97.9 KB


/tmp/ipykernel_37142/553117834.py:3: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df0 = pd.read_csv(csv_file0, parse_dates=['StartSession','EndSession'],sep='|').rename(columns={"Duartion": "Duration"})


In [58]:

df_time = pd.Timestamp(csv_file0.stem[9:], tz=current_timezone)
print(f'{df_time=}')
df_delta = datetime.now(tz=current_timezone) - df_time
df0['StartSession'].add(df_delta)

df_time=Timestamp('2024-01-01 00:10:00+0300', tz='Europe/Moscow')


0      2025-04-27 07:50:04.859693+03:00
1      2025-04-27 08:08:47.859693+03:00
2      2025-04-27 17:21:07.859693+03:00
3      2025-04-27 18:47:28.859693+03:00
4      2025-04-27 10:08:57.859693+03:00
                     ...               
1559   2025-04-27 18:41:39.859693+03:00
1560   2025-04-27 17:00:57.859693+03:00
1561   2025-04-27 16:03:10.859693+03:00
1562   2025-04-27 12:06:53.859693+03:00
1563   2025-04-27 18:06:17.859693+03:00
Name: StartSession, Length: 1564, dtype: datetime64[ns, Europe/Moscow]

In [59]:
from pydantic import BaseModel, Field

class Router(BaseModel):
    IdSession: int
    IdPSX: int
    IdSubscriber: int
    StartSession: datetime
    EndSesstion: datetime | None = None
    Duration: int
    UpTx: int
    DownTx: int

In [60]:
# df0.to_dict(orient='records')[0]
# df0['StartSession']
# Router(**df0.to_dict(orient='records')[0]).model_dump_json()

In [61]:
producer = get_kafka_producer()

2025-04-27 21:52:28.116 | INFO     | __main__:get_kafka_producer:15 - KafkaProducer успешно инициализирован!


In [62]:
# rec = Router(**df0.to_dict(orient='records')[0])
# send_rec(rec, producer=producer)

In [63]:
# for dict_rec in df0.to_dict(orient='records'):
#     rec = Router(**dict_rec)
#     producer.send(KAFKA_TOPIC, value=rec.model_dump_json())

In [75]:
import time

def wait_until(target_time: datetime):
    """Ждёт до указанного времени (формат 'HH:MM:SS')."""
    while True:
        now = datetime.now()
        if now >= target_time:
            break  # Если время уже наступило, выходим
        time_left = (target_time - now).total_seconds()
        time.sleep(min(time_left, 1))  # Ждём не более 1 секунды за раз

In [ ]:
def send_dataframe(dataframe: pd.DataFrame, model: BaseModel, producer: KafkaProducer, headers: list=None):
    for rec in dataframe.to_dict(orient='records'):
        producer.send(KAFKA_TOPIC, value=model(**rec).model_dump_json(), headers=headers)
    # producer.flush()

In [ ]:
start_time = datetime.now()
current_time = start_time

# Читаю сохраненую закладу времени
if Path(settings.time_pointer_file).exists():
    with open(Path(settings.time_pointer_file), 'rt') as time_pointer_file:
        current_time_pointer = pd.Timestamp(time_pointer_file.readline(), tz=current_timezone)
else:
    current_time_pointer = pd.Timestamp('1900-01-01 00:00:00', tz=current_timezone)

# Перебираем циклом каждый временной период по всем наборам источников за все время, выбираем одно время за раз
for i, (r0_file, r1_file, r2_file, r3_file, r4_file, r5_file) in enumerate(zip(r0_logs, r1_logs, r2_logs, r3_logs, r4_logs, r5_logs)):
    logger.info(f"Итерация: {i}, время: {current_time}")
    next_time = current_time + pd.Timedelta(minutes=1)

    # Берем поправку на время данных
    df_time = pd.Timestamp(r0_file.stem[9:], tz=current_timezone)
    df_delta = datetime.now(tz=current_timezone) - df_time

    # Пропускаем уже прочитанное время
    if df_time < current_time_pointer:
        logger.debug(f"Пропускаю время: {df_time.strftime('%Y-%m-%d %H:%M:%S')}")
        continue
    
    # берем txt источники
    for df_file in (r0_file, r1_file, r2_file):
        logger.debug(f"{df_file=}")
        df = pd.read_csv(df_file, sep='|', parse_dates=['StartSession','EndSession'], dayfirst=True).rename(columns={"Duartion": "Duration"})
        df['StartSession'] = df['StartSession'].dt.tz_localize('Etc/GMT-5').dt.tz_convert(current_timezone) + df_delta
        df['EndSession'] = df['EndSession'].dt.tz_localize('Etc/GMT-5').dt.tz_convert(current_timezone) + df_delta

        send_dataframe(dataframe=df, model=Router, producer=producer, headers=[('df_file', df_file.encode('utf8')),])
    # Берем csv источники
    for df_file in (r3_file, r4_file, r5_file):
        logger.debug(f"{df_file=}")
        df = pd.read_csv(df_file, sep=',', parse_dates=['StartSession','EndSession'], dayfirst=True).rename(columns={"Duartion": "Duration"})
        df['StartSession'] = df['StartSession'].dt.tz_localize('Etc/GMT-6').dt.tz_convert(current_timezone) + df_delta
        df['EndSession'] = df['EndSession'].dt.tz_localize('Etc/GMT-6').dt.tz_convert(current_timezone) + df_delta

        send_dataframe(dataframe=df, model=Router, producer=producer, headers=[('df_file', df_file.encode('utf8')),])
    producer.flush()

    # Записываем текущую временную метку
    with open(Path(settings.time_pointer_file), 'wt') as time_pointer_file:
        print((df_time + pd.Timedelta(minutes=10)).strftime('%Y-%m-%d %H:%M:%S'), file=time_pointer_file)

    # Ожидаем следующий перод времени
    wait_until(next_time)
    
    # Ограничиваем глубину вывода для отладки
    # if i > 2:
    #     break
        
    


2025-04-27 22:32:50.934 | INFO     | __main__:<module>:13 - Итерация: 0, время: 2025-04-27 22:32:50.932841
2025-04-27 22:32:50.938 | DEBUG    | __main__:<module>:22 - Пропускаю время: 2024-01-01 00:10:00
2025-04-27 22:32:50.939 | INFO     | __main__:<module>:13 - Итерация: 1, время: 2025-04-27 22:32:50.932841
2025-04-27 22:32:50.942 | DEBUG    | __main__:<module>:22 - Пропускаю время: 2024-01-01 00:20:00
2025-04-27 22:32:50.944 | INFO     | __main__:<module>:13 - Итерация: 2, время: 2025-04-27 22:32:50.932841
2025-04-27 22:32:50.945 | DEBUG    | __main__:<module>:22 - Пропускаю время: 2024-01-01 00:30:00
2025-04-27 22:32:50.946 | INFO     | __main__:<module>:13 - Итерация: 3, время: 2025-04-27 22:32:50.932841
2025-04-27 22:32:50.947 | DEBUG    | __main__:<module>:22 - Пропускаю время: 2024-01-01 00:40:00
2025-04-27 22:32:50.948 | INFO     | __main__:<module>:13 - Итерация: 4, время: 2025-04-27 22:32:50.932841
2025-04-27 22:32:50.950 | DEBUG    | __main__:<module>:27 - df_file=PosixPath